In [22]:
import pandas as pd
import datetime
from jobs import SRjobs

today = datetime.datetime.today()

preds = pd.read_csv('./SRoutput/weekly_preds/predictions_12_1_2018')
new_preds = pd.read_csv('./SRoutput/weekly_preds/predictions_12_1_2018_NF')

cols = ['XG', 'Keras', 'Keras_proba']
ext = new_preds[cols]
ext.columns = ['XG_new', 'K_new', 'Kpr_new']
preds = preds.join(ext)
preds['Date'] = pd.to_datetime(preds['Date'])
preds = preds[preds['Date'] < today]
preds.drop('Unnamed: 0', inplace=True, axis=1)
preds['Gamecode'] = preds.apply(lambda x: SRjobs.gamecode(x['Date'], x['Team'], x['Opp']), axis=1)

results = SRjobs.get_season_played_games()
results['Gamecode'] = results.apply(lambda x: SRjobs.gamecode(x['Date'], x['Team'], x['Opp']), axis=1)

Initiate get_season_played_games
Finished scraping 2018 schedule with 845 rows
Successfully loaded 844.0 played games from 2018


In [51]:
def wrong(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val == 'Wrong' else 'green'
    return 'background-color: %s' % color

def highlight_wrong(s,column):
    is_wrong = pd.Series(data=False, index=s.index)
    is_wrong[column] = s.loc[column] == 'Wrong'
    return ['background-color: lightcoral' if is_wrong.any() else 'background-color: lightgreen' for v in is_wrong]

preds['Results'] = 0
preds['Outcome'] = 0
for index, row in preds.iterrows():
    try:
        winner = results[results['Gamecode'] == row['Gamecode']]['Won'].values[0]
        preds['Results'].iloc[index] = winner
        if row['XG'] == winner:
            preds['Outcome'].iloc[index] = 'Correct'
        else:
            preds['Outcome'].iloc[index] = 'Wrong'
    except:
        preds['Results'].iloc[index] = 0
        if row['XG'] == 0:
            preds['Outcome'].iloc[index] = 'Correct'
        else:
            preds['Outcome'].iloc[index] = 'Wrong'
            
final_pred_check = preds.style.apply(highlight_wrong, column=['Outcome'], axis=1)
correct = 0
cnt = 0
for index, row in preds.iterrows():
    cnt += 1
    if row['Outcome'] == 'Correct':
        correct += 1
print('Accuracy: ', str(round(correct/cnt, 3)))
final_pred_check

Accuracy:  0.75


,Date,Team,Opp,XG,Keras,Keras_proba,XG_new,K_new,Kpr_new,Gamecode,Results,Outcome
0,2018-11-30 19:00:00,Buffalo,Northern Illinois,1,1,0.712283,1,1,0.698686,20181130BuffNort,0,Wrong
1,2018-11-30 20:00:00,Utah,Washington,0,0,0.479492,0,0,0.46029,20181130UtahWash,0,Correct
2,2018-12-01 12:00:00,Akron,South Carolina,0,0,0.199342,0,0,0.237715,2018121AkroSout,0,Correct
3,2018-12-01 16:00:00,Alabama,Georgia,1,1,0.780219,1,1,0.859625,2018121AlabGeor,1,Correct
4,2018-12-01 13:30:00,Alabama-Birmingham,Middle Tennessee State,0,0,0.249796,0,0,0.268924,2018121AlabMidd,1,Wrong
5,2018-12-01 20:00:00,Clemson,Pittsburgh,1,1,0.957843,1,1,0.970244,2018121ClemPitt,1,Correct
6,2018-12-01 12:00:00,Drake,Iowa State,0,0,0.0394571,0,0,0.0593184,2018121DrakIowa,0,Correct
7,2018-12-01 12:00:00,East Carolina,North Carolina State,0,0,0.11817,0,0,0.199156,2018121EastNort,0,Correct
8,2018-12-01 19:45:00,Fresno State,Boise State,0,0,0.332354,0,0,0.407805,2018121FresBois,1,Wrong
9,2018-12-01 12:00:00,Louisiana,Appalachian State,0,0,0.291955,0,0,0.343365,2018121LouiAppa,0,Correct
